In [2]:
import scanpy as sc
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder 
import torch
from sys import getsizeof

import os, sys
sys.path.append('../src')
from models.lib.data import *
labelfile = '../data/mouse/Adult Inhibitory Neurons in Mouse_labels.tsv'

label_df = pd.read_csv(labelfile, sep='\t')
labels = label_df.loc[:, 'numeric_class']
labels

0         36
1         36
2          9
3         40
4         40
          ..
141064    36
141065    41
141066    41
141067    36
141068    36
Name: numeric_class, Length: 141069, dtype: int64

In [3]:
mouse_atlas = sc.read_h5ad('../data/mouse/MouseAdultInhibitoryNeurons.h5ad')
mo_data = sc.read_h5ad('../data/mouse/Mo_PV_paper_TDTomato_mouseonly.h5ad')

Variable names are not unique. To make them unique, call `.var_names_make_unique`.


In [4]:
trainsplit, valsplit = train_test_split(labels, stratify=labels)

In [5]:
trainsplit, valsplit

(45020     34
 86509     27
 91585     26
 115080    42
 20776      0
           ..
 39831     43
 101474    24
 134026    20
 88176     44
 39507     43
 Name: numeric_class, Length: 105801, dtype: int64,
 114477     2
 49996     20
 54321     15
 121414    24
 30482      7
           ..
 22782      0
 115924    45
 81482     27
 54441      5
 65732      9
 Name: numeric_class, Length: 35268, dtype: int64)

In [10]:
label_df.loc[trainsplit, :]

,cellId,dataset_name,batch_name,timepoint,region,class,hires_leiden,leiden,Expressed Genes,latent_cell_probability,latent_time,UMI Count,cell,numeric_class
31,GCAATCGAGCTGAT_p25-27_Amygdala_SAMN08730984,SRP135960_linnarson_adultmouse,p25-27_Amygdala_SAMN08730984,46.0,amygdala,Amy/Hypo_HAP1/PEG10,26,26,995,0.999715,0.401222,1848.0,31,0
36,TCAGCGCTGTCATG_p25-27_Amygdala_SAMN08730984,SRP135960_linnarson_adultmouse,p25-27_Amygdala_SAMN08730984,46.0,amygdala,Glia,51,51,985,0.999758,0.314352,1734.0,36,18
38,TGAACCGATGACCA_p25-27_Amygdala_SAMN08730984,SRP135960_linnarson_adultmouse,p25-27_Amygdala_SAMN08730984,46.0,amygdala,Amy/Hypo_HAP1/PEG10,6,6,902,0.999747,0.370432,1500.0,38,0
2,GACCAAACTGCCTC_p25-27_Amygdala_SAMN08730984,SRP135960_linnarson_adultmouse,p25-27_Amygdala_SAMN08730984,46.0,amygdala,Ctx_LHX6/SST,67,67,2288,0.999920,0.532447,5297.0,2,9
27,ATATAGTGACCCAA_p25-27_Amygdala_SAMN08730984,SRP135960_linnarson_adultmouse,p25-27_Amygdala_SAMN08730984,46.0,amygdala,Ctx_PVALB/VIPR2,114,114,974,0.999835,0.426167,1766.0,27,13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1,GAAACCCAATCTCG_p25-27_Amygdala_SAMN08730984,SRP135960_linnarson_adultmouse,p25-27_Amygdala_SAMN08730984,46.0,amygdala,S-phase_MCM4/H43C,82,82,2897,0.999931,0.302482,5239.0,1,36
27,ATATAGTGACCCAA_p25-27_Amygdala_SAMN08730984,SRP135960_linnarson_adultmouse,p25-27_Amygdala_SAMN08730984,46.0,amygdala,Ctx_PVALB/VIPR2,114,114,974,0.999835,0.426167,1766.0,27,13
19,CACTATACAACGGG_p25-27_Amygdala_SAMN08730984,SRP135960_linnarson_adultmouse,p25-27_Amygdala_SAMN08730984,46.0,amygdala,Amy/Hypo_HAP1/PEG10,3,3,1235,0.999793,0.394735,2297.0,19,0
1,GAAACCCAATCTCG_p25-27_Amygdala_SAMN08730984,SRP135960_linnarson_adultmouse,p25-27_Amygdala_SAMN08730984,46.0,amygdala,S-phase_MCM4/H43C,82,82,2897,0.999931,0.302482,5239.0,1,36


In [22]:
genes = [x.upper() for x in mouse_atlas.var['name-0-10'].values]
mogenes = [x.upper() for x in mo_data.var.index]
refgenes = sorted(list(set(mogenes).intersection(genes)))

In [11]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(mouse_atlas.X, labels, stratify=labels)
X_train, y_train

(array([[-0.04272555, -0.00983691, -0.51307064, ..., -0.01356186,
         -0.01273745, -0.00330662],
        [-0.04272555, -0.00983691, -0.51307064, ..., -0.01356186,
         -0.01273745, -0.00330662],
        [-0.04272555, -0.00983691, -0.51307064, ..., -0.01356186,
         -0.01273745, -0.00330662],
        ...,
        [-0.04272555, -0.00983691, -0.51307064, ..., -0.01356186,
         -0.01273745, -0.00330662],
        [-0.04272555, -0.00983691, -0.51307064, ..., -0.01356186,
         -0.01273745, -0.00330662],
        [-0.04272555, -0.00983691, -0.51307064, ..., -0.01356186,
         -0.01273745, -0.00330662]], dtype=float32),
 91709      2
 137740    27
 84323     23
 133109    36
 79166     27
           ..
 2039       9
 43008     36
 69201      9
 41816     36
 92441     41
 Name: numeric_class, Length: 105801, dtype: int64)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(mouse_atlas.X, labels.values, stratify=labels.values)

X_train, y_train

In [10]:
atlas_train = NumpyStream(
    matrix=X_train,
    labels=y_train,
    sep='\t',
    columns=list(mouse_atlas.var.index),
)

In [11]:
atlas_val = NumpyStream(
    matrix = X_test,
    labels = y_test,
    sep='\t',
    columns=list(mouse_atlas.var.index),
    class_label='ingore'
)

In [12]:
len(atlas_val)

35268

In [13]:
train = DataLoader(atlas_train, batch_size=4, num_workers=0, drop_last=True, shuffle=True)
val = DataLoader(atlas_val, batch_size=4, num_workers=0, drop_last=True, shuffle=False)

In [ ]:
from src.models.lib.neural import GeneClassifier
from pytorch_lightning import Trainer 
from pytorch_lightning.loggers import WandbLogger

model = GeneClassifier(
    input_dim=34430, 
    output_dim=50,
    optim_params={
        'optimizer': torch.optim.Adam,
        'lr': 3e-4,
    }
)

wandb_logger = WandbLogger(project='Mouse Classifier', name='Tabnet with Metrics')
trainer = Trainer(logger=wandb_logger)

trainer.fit(model, train, val)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


Model initialized. input_dim = 34430, output_dim = 50. Metrics are dict_keys(['accuracy', 'precision', 'recall']) and weighted_metrics = False


wandb: Currently logged in as: jlehrer1 (use `wandb login --relogin` to force relogin)



  | Name       | Type                 | Params
----------------------------------------------------
0 | base_model | TabNetGeneClassifier | 2.3 M 
----------------------------------------------------
2.3 M     Trainable params
0         Non-trainable params
2.3 M     Total params
9.114     Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

wandb: Network error (ConnectionError), entering retry loop.


Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

In [ ]:
def gene_intersection(
    files
):
    import dask.dataframe as dd 
    
    cols = []
    for file in files:
        temp = pd.read_csv(fpath, nrows=1, header=1).columns 
        temp = [x.split('|')[0].upper().strip() for x in temp]
        cols.append(set(temp))
    
    unique = list(set.intersection(*cols))
    unique = sorted(unique)
    
    return unique 

In [17]:
import torch
from sklearn.model_selection import train_test_split 
import pandas as pd 
import pathlib 
import warnings 

def generate_single_dataset(
    datafile: str,
    labelfile: str,
    class_label: str,
    test_prop=0.2,
    sep=',',
    *args,
    **kwargs,
):
    current_labels = pd.read_csv(labelfile, sep=sep).loc[:, class_label]
    
    suffix = pathlib.Path(datafile).suffix
    print(f'{suffix = }')
    if suffix == '.h5ad':
        data = sc.read_h5ad(datafile)
        X_train, X_test, y_train, y_test = train_test_split(data, current_labels.values, stratify=current_labels.values, test_size=test_prop)

        X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, stratify=y_train, test_size=test_prop) # 0.25 x 0.8 = 0.2\
        
        train, val, test = (
            NumpyStream(
                matrix=matrix,
                labels=labels,
                class_label=class_label,
                *args,
                **kwargs,
            )
            for matrix, labels in zip([X_train, X_val, X_test], [y_train, y_val, y_test])
        )
        
    else:
        if suffix != '.csv' or suffix != '.tsv':
            warnings.warn(f'Extension {suffix} not recognized, \
                interpreting as .csv. To silence this warning, pass in explicit file types.')
    
        # Make stratified split on labels
        trainsplit, valsplit = train_test_split(current_labels, stratify=current_labels, test_size=test_prop)
        trainsplit, testsplit = train_test_split(trainsplit, stratify=trainsplit, test_size=test_prop)

        train, val, test = (
            GeneExpressionData(
                filename=datafile,
                labelname=labelfile,
                class_label=class_label,
                indices=indices,
                *args,
                **kwargs,
            )
            for indices in [trainsplit, valsplit, testsplit]  
        )

    return train, val, test 


In [ ]:
generate_single_dataset(
    datafile='../data/mouse/MouseAdultInhibitoryNeurons.h5ad',
    labelfile='../data/mouse/Adult Inhibitory Neurons in Mouse_labels.tsv',
    class_label='numeric_class',
    sep='\t'
)

suffix = '.h5ad'
